# Imports

In [ ]:
#import detectron2
import os, cv2
import torch
import glob

In [ ]:
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

# Setup Model

In [ ]:
cfg = get_cfg()
cfg.MODEL.DEVICE = 'cpu'
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

# Predict

In [ ]:
predict_paths = glob.glob("../../test_images/rcf/*.jpg")

In [ ]:
images = [cv2.imread(path) for path in predict_paths]

In [ ]:
cfg = get_cfg()
cfg.MODEL.DEVICE = 'cpu'
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
#Load model. The OUTPUT_DIR should be the outputs/detectron2/rcf folder. Otherwise change the path.
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "../../../models/detectron2/Detectron2RCF.pth")

In [ ]:
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4
predictor = DefaultPredictor(cfg)

# Zeitmessung

In [ ]:
def prediction_wrapper(paths_to_images):
    images = [cv2.imread(path) for path in paths_to_images]
    for i in range(0, len(images)):
        output = predictor(images[i])["instances"].pred_masks.cpu()[0]*255
        torch.save(output , cfg.OUTPUT_DIR + "/timetest/inference" + str(i) + ".pt")

In [ ]:
%%timeit -r 10 -n 1
    prediction_wrapper(predict_paths)